In [25]:
import math

def get_roll(face_landmarks):
    left_eye_center_x = (face_landmarks[133].x - face_landmarks[33].x)/2
    left_eye_center_y = (face_landmarks[133].y - face_landmarks[33].y)/2
    right_eye_center_x = (face_landmarks[362].x - face_landmarks[263].x)/2
    right_eye_center_y = (face_landmarks[362].y - face_landmarks[263].y)/2
    dx = abs(right_eye_center_x - left_eye_center_x)
    dy = abs(right_eye_center_y - left_eye_center_y)

    # Calculate roll in degrees
    roll = abs(math.atan2(dy, dx)) / math.pi * 180
    return roll

In [26]:
def get_pitch(face_landmarks):
    nose_y = face_landmarks[1].y
    eye_y = (face_landmarks[33].y + face_landmarks[263].y)/2
    mouth_y = face_landmarks[0].y
    pitch_measure = abs(eye_y - nose_y) / abs(mouth_y - nose_y)
    pitch_down = max(pitch_measure - 3.0, 0) / 4.5 * 45
    pitch_up = max(1 / pitch_measure - 0.9, 0) / 0.8 * 45

    return max(pitch_down, pitch_up) 

# pd => 3.0 => 0, 7.5 => 45
# pu => 0.9 => 0, 1.7 => 45

In [27]:
def get_yaw(face_landmarks):
    left_dist = abs(face_landmarks[130].x - face_landmarks[127].x)
    right_dist = abs(face_landmarks[359].x - face_landmarks[356].x)
    face_width = abs(face_landmarks[356].x - face_landmarks[127].x)
    return (abs(right_dist - left_dist)/face_width)/0.05 * 8

In [7]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

# STEP 2: Create an FaceLandmarker object.
def get_landmarks(image_path):
    base_options = python.BaseOptions(model_asset_path='../utilities/face_landmarker.task')
    options = vision.FaceLandmarkerOptions(base_options=base_options,
                                        output_face_blendshapes=True,
                                        output_facial_transformation_matrixes=True,
                                        num_faces=1)
    detector = vision.FaceLandmarker.create_from_options(options)

    # STEP 3: Load the input image
    frame = cv2.imread(image_path)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(image)

    face_landmarks_list = detection_result.face_landmarks
    face_landmarks = face_landmarks_list[0]
    return face_landmarks

In [30]:
import glob
import pandas as pd
from datetime import datetime

all_images = glob.glob("D:/oneid_backend/icao-guidelines-server/dataset/*")

In [5]:
def logger(image, is_valid, path="./"):
    time_str = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    df_headers = pd.DataFrame({
        'log_time': [time_str],
        'image': [image],
        'is_valid': [is_valid],
    })
    df_headers.to_csv(f'{path}roll_pitch_yaw_test.csv', mode='a', index=False, header=False)

In [31]:
# time duration
start_time = datetime.now()

for image_path in all_images:
    landmarks = get_landmarks(image_path)
    roll = get_roll(landmarks)
    yaw = get_yaw(landmarks)
    pitch = get_pitch(landmarks)
    avg = (roll + yaw + pitch) / 3
    is_valid = avg < 8
    logger(image_path, is_valid)
    print(image_path, is_valid)

D:/oneid_backend/icao-guidelines-server/dataset\N230101660.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101661.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101664.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101669.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101671.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101683.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101691.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101696.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101709.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101710.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101717.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101719.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101726.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\N230101743.JPG True
D:/oneid_backend/icao-guidelines-server/dataset\